In [1]:
from src.dsl_research_assistant.captions.image_captioning import ImageCaptioning
import os

c:\Users\grado\Desktop\dsl-research-assistant\backend\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_names_tasks = [ ("xtuner/llava-llama-3-8b-v1_1-transformers", "image-text-to-text")]  #"mPLUG/DocOwl2", "ds4sd/SmolDocling-256M-preview", ("Salesforce/blip-image-captioning-base","image-to-text"),

In [4]:
image_captioning = ImageCaptioning(model_name="Salesforce/blip-image-captioning-base", task="image-to-text")
image_path = "C:\\Users\\grado\\Desktop\\dsl-research-assistant\\backend\\image_captions\\Abdal_Gaussian_Shell_Maps_for_Efficient_3D_Human_Generation_CVPR_2024_paper\\Abdal_Gaussian_Shell_Maps_for_Efficient_3D_Human_Generation_CVPR_2024_paper-picture-2.png"
prompt = "Describe this image in a few sentences."
context = "This CVPR paper is the Open Access version, provided by the Computer Vision Foundation. Except for this watermark, it is identical to the accepted version; the final published version of the proceedings is available on IEEE Xplore. Gaussian Shell Maps for Efficient 3D Human Generation Rameen Abdal *1 Wang Yifan *1 Zifan Shi *†1,2 Yinghao Xu 1 Ryan Po 1 Zhengfei Kuang 1 Qifeng Chen 2 Dit-Yan Yeung 2 Gordon Wetzstein 1 1 Stanford University, USA HKUST, Hong Kong 2"
print("Generating caption for the image...")
caption = image_captioning.generate_caption(image_path, prompt, context)
print(f"Generated caption: {caption}")

Device set to use cuda


Generating caption for the image...
Generated caption: a group of women standing in different poses


In [3]:
import os
from glob import glob
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
import evaluate
from pycocoevalcap.cider.cider import Cider

cider_scorer = Cider()
base_dir = "C:\\Users\\grado\\Desktop\\dsl-research-assistant\\backend\\image_captions"
#image_captioning = ImageCaptioning(model_name="ds4sd/SmolDocling-256M-preview", task="image-text-to-text")
image_captioning = ImageCaptioning(model_name="Salesforce/blip-image-captioning-base", task="image-to-text")
blue_scores = []
print(f"Starting evaluation in directory: {base_dir}")
for subdir, dirs, files in os.walk(base_dir):
    print(f"Processing directory: {subdir}")
    if "caption.txt" in files:
        caption_file = os.path.join(subdir, "caption.txt")
        with open(caption_file, "r", encoding="utf-8") as f:
            gt_captions = [line.strip() for line in f.readlines() if not line.startswith("__")]
        image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        for idx, image_file in enumerate(image_files):
            image_path = os.path.join(subdir, image_file)
            gt_caption = gt_captions[idx] if idx < len(gt_captions) else ""
            if not gt_caption:
                print(f"No ground truth caption found for {image_file}. Skipping...")
                continue
            prompt = "Describe this image in a few sentences."
            context = ""
            generated_caption = image_captioning.generate_caption(image_path, prompt, context)
            bleu = sentence_bleu([gt_caption.split()], generated_caption.split())
            rouge = Rouge()
            rouge_scores = rouge.get_scores(generated_caption, gt_caption)[0]
            gts = {0: [gt_caption]}
            res = {0: [generated_caption]}
            cider_score, _ = cider_scorer.compute_score(gts, res)
            print(f"Image: {image_file}")
            print(f"GT Caption: {gt_caption}")
            print(f"Generated: {generated_caption}")
            print(f"BLEU Score: {bleu:.4f}")
            print(f"ROUGE Scores: {rouge_scores}\n")
            print(f"CIDEr Score: {cider_score:.4f}\n")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


Starting evaluation in directory: C:\Users\grado\Desktop\dsl-research-assistant\backend\image_captions
Processing directory: C:\Users\grado\Desktop\dsl-research-assistant\backend\image_captions
Processing directory: C:\Users\grado\Desktop\dsl-research-assistant\backend\image_captions\Abdal_Gaussian_Shell_Maps_for_Efficient_3D_Human_Generation_CVPR_2024_paper
No ground truth caption found for Abdal_Gaussian_Shell_Maps_for_Efficient_3D_Human_Generation_CVPR_2024_paper-picture-1.png. Skipping...


c:\Users\grado\Desktop\dsl-research-assistant\backend\.venv\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\grado\Desktop\dsl-research-assistant\backend\.venv\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\grado\Desktop\dsl-research-assistant\backend\.venv\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram o

Image: Abdal_Gaussian_Shell_Maps_for_Efficient_3D_Human_Generation_CVPR_2024_paper-picture-10.png
GT Caption: Gaussian Shell Maps. Gaussian Shell Maps is an efficient framework for 3D human generation connecting 3D Gaussians with CNN-based generators. 3D Gaussians are anchored to 'shells' derived from the SMPL template [37] (only two shells are visualized for clarity), and the appearance is modeled in texture space. Trained only on 2D images, we show that our method can generate diverse articulable humans in real-time with state-of-the-art quality directly in high resolution without the need for upsampling and hence avoiding aliasing artifacts.
Generated: a diagram showing the different types of the body
BLEU Score: 0.0000
ROUGE Scores: {'rouge-1': {'r': 0.015384615384615385, 'p': 0.125, 'f': 0.02739725832238708}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.015384615384615385, 'p': 0.125, 'f': 0.02739725832238708}}

CIDEr Score: 0.0000

Image: Abdal_Gaussian_Shell_Map

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Image: Abdelfattah_MaskCLR_Attention-Guided_Contrastive_Learning_for_Robust_Action_Representation_Learning_CVPR_2024_paper-picture-11.png
GT Caption: (b) Accuracy under Gaussian noise N (0 , 0 002 . 2 ) .
Generated: the graph of the line of the curve is shown in the diagram
BLEU Score: 0.0000
ROUGE Scores: {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}

CIDEr Score: 0.0000

No ground truth caption found for Abdelfattah_MaskCLR_Attention-Guided_Contrastive_Learning_for_Robust_Action_Representation_Learning_CVPR_2024_paper-picture-12.png. Skipping...


KeyboardInterrupt: 

In [7]:
print("Average BLEU Score:", sum(blue_scores) / len(blue_scores) if blue_scores else 0)
print("Average ROUGE Scores:", rouge_scores if 'rouge_scores' in locals() else "No scores computed")

Average BLEU Score: 0
Average ROUGE Scores: {'rouge-1': {'r': 0.02127659574468085, 'p': 0.1, 'f': 0.035087716405047945}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.02127659574468085, 'p': 0.1, 'f': 0.035087716405047945}}
